In [1]:
FN = 'train'

you should use GPU but if it is busy then you always can fall back to your CPU

In [2]:
import os
# os.environ['THEANO_FLAGS'] = 'device=cpu,floatX=float32'

In [3]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

Use indexing of tokens from [vocabulary-embedding](./vocabulary-embedding.ipynb) this does not clip the indexes of the words to `vocab_size`.

Use the index of outside words to replace them with several `oov` words (`oov` , `oov0`, `oov1`, ...) that appear in the same description and headline. This will allow headline generator to replace the oov with the same word in the description

In [4]:
FN0 = 'vocabulary-embedding'

implement the "simple" model from http://arxiv.org/pdf/1512.01712v1.pdf

you can start training from a pre-existing model. This allows you to run this notebooks many times, each time using different parameters and passing the end result of one run to be the input of the next.

I've started with `maxlend=0` (see below) in which the description was ignored. I then moved to start with a high `LR` and the manually lowering it. I also started with `nflips=0` in which the original headlines is used as-is and slowely moved to `12` in which half the input headline was fliped with the predictions made by the model (the paper used fixed 10%)

In [5]:
FN1 = 'train'

input data (`X`) is made from `maxlend` description words followed by `eos`
followed by headline words followed by `eos`
if description is shorter than `maxlend` it will be left padded with `empty`
if entire data is longer than `maxlen` it will be clipped and if it is shorter it will be right padded with empty.

labels (`Y`) are the headline words followed by `eos` and clipped or padded to `maxlenh`

In other words the input is made from a `maxlend` half in which the description is padded from the left
and a `maxlenh` half in which `eos` is followed by a headline followed by another `eos` if there is enough space.

The labels match only the second half and 
the first label matches the `eos` at the start of the second half (following the description in the first half)

In [6]:
maxlend=25 # 0 - if we dont want to use description at all
maxlenh=25
maxlen = maxlend + maxlenh
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False

the out of the first `activation_rnn_size` nodes from the top LSTM layer will be used for activation and the rest will be used to select predicted word

In [7]:
activation_rnn_size = 40 if maxlend else 0

In [8]:
# training parameters
seed=42
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=64
nflips=10

In [9]:
nb_train_samples = 30000
nb_val_samples = 3000

# read word embedding

In [10]:
import pickle as pickle

with open('/token/token2.pkl'.format(FN0), 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
vocab_size, embedding_size = embedding.shape

In [11]:
with open('/token/token3.pkl'.format(FN0), 'rb') as fp:
    X, Y = pickle.load(fp)

In [12]:
nb_unknown_words = 10

In [13]:
print('number of examples',len(X),len(Y))
print('dimension of embedding space for words',embedding_size)
print('vocabulary size', vocab_size, 'the last {} words can be used as place holders for unknown/oov words'.format(nb_unknown_words))
print('total number of different words',len(idx2word), len(word2idx))
print('number of words outside vocabulary which we can substitue using glove similarity', len(glove_idx2idx))
print('number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(glove_idx2idx))

number of examples 42570 42570
dimension of embedding space for words 100
vocabulary size 604571 the last 10 words can be used as place holders for unknown/oov words
total number of different words 604573 604573
number of words outside vocabulary which we can substitue using glove similarity 5
number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov) -3


In [14]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

when printing mark words outside vocabulary with `^` at their end

In [15]:
oov0 = vocab_size-nb_unknown_words

In [16]:
for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [17]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(39570, 39570, 3000, 3000)

In [18]:
del X
del Y

In [19]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [20]:
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys

In [21]:
def prt(label, x):
    print(label+':',)
    for w in x:
        print(idx2word[w],)
    print()

In [22]:
i = 334
prt('H',Y_train[i])
prt('D',X_train[i])

H:
With
Sessions
Confirmed,
Alabama
Governor
Appoints
New
Senator

D:
Luther
Strange
will
go
from
being
Alabama’s
attorney
general
to
being
the
state’s
junior
senator,
as
Gov.
Robert
Bentley
says
he
will
appoint
Strange
to
the
seat
vacated
by
Sen.
Jeff
Sessions
—
who’s
slated
to
be
sworn
in
as
the
new
U.
S.
attorney
general
Thursday
morning.
”I
am
greatly
honored
and
humbled
to
accept
the
appointment
to
Alabama’s
Senate
seat
vacated
by
Senator
Jeff
Sessions,”
Strange
said
in
a
news
release
from
the
governor’s
office.
”Senator
Sessions’
commitment
to
public
service
is
nearly
unparalleled
in
Alabama
history
and
his
departure
from
the
Senate
leaves
tremendous
shoes
to
fill.”
In
choosing
Strange,
Bentley
is
making
the
pick
many
had
expected.
But
the
situation
has
also
been
a
topic
of
talk
both
inside
and
outside
of
Alabama,
because
Strange
has
worked
on
several
prominent
legal
cases
—
including
an
inquiry
into
allegations
that
Bentley
had
an
affair
with
an
aide
and
possibly
used
state
fund

In [23]:
i = 334
prt('H',Y_test[i])
prt('D',X_test[i])

H:
Why
misogyny
won

D:
After
leaked
audio
showed
Donald
Trump
bragging
in
2005
that
he
can
“grab
[women]
by
the
pussy”
and
kiss
them
without
consent
because
he’s
“a
star,”
Trump’s
campaign
seemed
done
for.
Many
Republicans
withdrew
their
endorsements
in
disgust,
and
those
who
didn’t
faced
intense
pressure
to
follow
suit.
Trump’s
poll
numbers
plummeted
—
and
kept
plummeting
after
women
started
coming
forward
to
allege
that
Trump
had
sexually
assaulted
them.
But
then,
the
free
fall
stopped.
Media
attention
turned
back
to
Hillary
Clinton’s
emails
with
a
little
over
a
week
to
go
before
the
election.
A
new
accusation
against
Trump
from
a
former
Miss
Finland,
and
a
newly
surfaced
video
that
showed
Trump
grabbing
and
kissing
a
former
Miss
Universe
after
humiliating
her
onstage
in
front
of
thousands,
barely
caused
a
ripple.
And
then
Americans
elected
an
alleged
sexual
predator
to
be
their
president.
They
chose
a
man
who
has
now
been
accused
of
sexual
assault
by
15
women
—
a
man
who
has
promis

# Model

In [24]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers import Merge
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2

In [25]:
# seed weight initialization
random.seed(seed)
np.random.seed(seed)

In [26]:
regularizer = l2(weight_decay) if weight_decay else None

start with a standard stacked LSTM

In [27]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    W_regularizer=regularizer, dropout=p_emb, weights=[embedding], mask_zero=True,
                    name='embedding_1'))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True, # batch_norm=batch_norm,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(604571, 100, input_length=50, weights=[array([[-..., mask_zero=True, name="embedding_1", embeddings_regularizer=None)`
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, name="lstm_1", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequenc

A special layer that reduces the input just to its headline part (second half).
For each word in this part it concatenate the output of the previous layer (RNN)
with a weighted average of the outputs of the description part.
In this only the last `rnn_size - activation_rnn_size` are used from each output.
The first `activation_rnn_size` output is used to computer the weights for the averaging.

In [28]:
from keras.layers.core import Lambda
import keras.backend as K

def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend,:], X[:,maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    activation_energies = activation_energies + -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_avg_word, head_words))


class SimpleContext(Lambda):
    def __init__(self,**kwargs):
        super(SimpleContext, self).__init__(simple_context,**kwargs)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def compute_output_shape(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [29]:
if activation_rnn_size:
    model.add(SimpleContext(name='simplecontext_1'))
model.add(TimeDistributed(Dense(vocab_size,W_regularizer=regularizer, b_regularizer=regularizer,name = 'timedistributed_1')))
model.add(Activation('softmax', name='activation_1'))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(604571, name="timedistributed_1", kernel_regularizer=None, bias_regularizer=None)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop
# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [31]:
%%javascript
// new Audio("http://www.soundjay.com/button/beep-09.wav").play ()

<IPython.core.display.Javascript object>

In [32]:
K.set_value(model.optimizer.lr,np.float32(LR))

In [33]:
def str_shape(x):
    return 'x'.join(map(str,x.shape))
    
def inspect_model(model):
    for i,l in enumerate(model.layers):
        print(i, 'cls=%s name=%s'%(type(l).__name__, l.name))
        weights = l.get_weights()
        for weight in weights:
            print(str_shape(weight),)
        print()

In [34]:
inspect_model(model)

0 cls=Embedding name=embedding_1
604571x100

1 cls=LSTM name=lstm_1
100x2048
512x2048
2048

2 cls=Dropout name=dropout_1

3 cls=LSTM name=lstm_2
512x2048
512x2048
2048

4 cls=Dropout name=dropout_2

5 cls=LSTM name=lstm_3
512x2048
512x2048
2048

6 cls=Dropout name=dropout_3

7 cls=SimpleContext name=simplecontext_1

8 cls=TimeDistributed name=time_distributed_1
944x604571
604571

9 cls=Activation name=activation_1



# Load

In [35]:
if FN1 and os.path.exists('/output/data.hdf5'):
    model.load_weights('/output/data/%s.hdf5')

# Test

In [36]:
def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

In [37]:
samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [38]:
np.all(data[:,maxlend] == eos)

True

In [39]:
data.shape,map(len, samples)

((1, 50), <map at 0x7f0af460d198>)

In [40]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

(1, 25, 604571)

# Sample generation

this section is only used to generate examples. you can skip it if you just want to understand how the training works

In [41]:
# variation to https://github.com/ryankiros/skip-thoughts/blob/master/decoding/search.py
def beamsearch(predict, start=[empty]*maxlend + [eos],
               k=1, maxsample=maxlen, use_unk=True, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in range(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [list(start)]
    live_scores = [0]

    while live_k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk:
            for i in range(nb_unknown_words):
                cand_scores[:,vocab_size - 1 - i] = 1e20
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        ranks_dead = [r for r in ranks if r < n]
        ranks_live = [r - n for r in ranks if r >= n]
        
        dead_scores = [dead_scores[r] for r in ranks_dead]
        dead_samples = [dead_samples[r] for r in ranks_dead]
        
        live_scores = [live_scores[r] for r in ranks_live]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_live]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        zombie = [s[-1] == eos or len(s) > maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)

    return dead_samples + live_samples, dead_scores + live_scores

In [42]:
# !pip install python-Levenshtein

In [43]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = map(len, samples)
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([prob[sample_length-maxlend-1] for prob, sample_length in zip(probs, sample_lengths)])

In [44]:
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < oov0 else glove_idx2idx.get(x,x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= oov0])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    return xs

In [45]:
def vocab_unfold(desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= oov0:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]

In [46]:
import sys
import Levenshtein

def gensamples(skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    i = random.randint(0,len(X_test)-1)
    print('HEAD:',' '.join(idx2word[w] for w in Y_test[i][:maxlenh]))
    print('DESC:',' '.join(idx2word[w] for w in X_test[i][:maxlend]))
    sys.stdout.flush()

    print('HEADS:')
    x = X_test[i]
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in zip(sample,score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code,c) for c in codes])
            if distance > -0.6:
                print(score, ' '.join(words))
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print(score, ' '.join(words))
        codes.append(code)

In [47]:
import numpy
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEAD: Trump presses China on North Korea ahead of Xi talks
DESC: U. S. President Donald Trump held out the possibility on Sunday of using trade as a lever to secure Chinese cooperation against North Korea and
HEADS:


IndexError: index 0 is out of bounds for axis 1 with size 0

# Data generator

Data generator generates batches of inputs and outputs/labels for training. The inputs are each made from two parts. The first maxlend words are the original description, followed by `eos` followed by the headline which we want to predict, except for the last word in the headline which is always `eos` and then `empty` padding until `maxlen` words.

For each, input, the output is the headline words (without the start `eos` but with the ending `eos`) padded with `empty` words up to `maxlenh` words. The output is also expanded to be y-hot encoding of each word.

To be more realistic, the second part of the input should be the result of generation and not the original headline.
Instead we will flip just `nflips` words to be from the generator, but even this is too hard and instead
implement flipping in a naive way (which consumes less time.) Using the full input (description + eos + headline) generate predictions for outputs. For nflips random words from the output, replace the original word with the word with highest probability from the prediction.

In [48]:
def flip_headline(x, nflips=None, model=None, debug=False):
    """given a vectorized input (after `pad_sequences`) flip some of the words in the second half (headline)
    with words predicted by the model
    """
    if nflips is None or model is None or nflips <= 0:
        return x
    
    batch_size = len(x)
    assert np.all(x[:,maxlend] == eos)
    probs = model.predict(x, verbose=0, batch_size=batch_size)
    x_out = x.copy()
    for b in range(batch_size):
        # pick locations we want to flip
        # 0...maxlend-1 are descriptions and should be fixed
        # maxlend is eos and should be fixed
        flips = sorted(random.sample(range(maxlend+1,maxlen), nflips))
        if debug and b < debug:
            print(b,)
        for input_idx in flips:
            if x[b,input_idx] == empty or x[b,input_idx] == eos:
                continue
            # convert from input location to label location
            # the output at maxlend (when input is eos) is feed as input at maxlend+1
            label_idx = input_idx - (maxlend+1)
            prob = probs[b, label_idx]
            w = prob.argmax()
            if w == empty:  # replace accidental empty with oov
                w = oov0
            if debug and b < debug:
                print('{} => {}'.format(idx2word[x_out[b,input_idx]],idx2word[w]),)
            x_out[b,input_idx] = w
        if debug and b < debug:
            print()
    return x_out

In [49]:
def conv_seq_labels(xds, xhs, nflips=None, model=None, debug=False):
    """description and hedlines are converted to padded input vectors. headlines are one-hot to label"""
    batch_size = len(xhs)
    assert len(xds) == batch_size
    x = [vocab_fold(lpadd(xd)+xh) for xd,xh in zip(xds,xhs)]  # the input does not have 2nd eos
    x = sequence.pad_sequences(x, maxlen=maxlen, value=empty, padding='post', truncating='post')
    x = flip_headline(x, nflips=nflips, model=model, debug=debug)
    
    y = np.zeros((batch_size, maxlenh, vocab_size))
    for i, xh in enumerate(xhs):
        xh = vocab_fold(xh) + [eos] + [empty]*maxlenh  # output does have a eos at end
        xh = xh[:maxlenh]
        y[i,:,:] = np_utils.to_categorical(xh, vocab_size)
        
    return x, y

In [50]:
def gen(Xd, Xh, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, seed=seed):
    """yield batches. for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    
    while training it is good idea to flip once in a while the values of the headlines from the
    value taken from Xh to value generated by the model.
    """
    c = nb_batches if nb_batches else 0
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
        new_seed = random.randint(0, sys.maxsize)
        random.seed(c+123456789+seed)
        for b in range(batch_size):
            t = random.randint(0,len(Xd)-1)

            xd = Xd[t]
            s = random.randint(min(maxlend,len(xd)), max(maxlend,len(xd)))
            xds.append(xd[:s])
            
            xh = Xh[t]
            s = random.randint(min(maxlenh,len(xh)), max(maxlenh,len(xh)))
            xhs.append(xh[:s])

        # undo the seeding before we yield inorder not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(xds, xhs, nflips=nflips, model=model, debug=debug)

In [51]:
r = next(gen(X_train, Y_train, batch_size=batch_size))
r[0].shape, r[1].shape, len(r)

((64, 50), (64, 25, 604571), 2)

In [52]:
def test_gen(gen, n=5):
    Xtr,Ytr = next(gen)
    for i in range(n):
        assert Xtr[i,maxlend] == eos
        x = Xtr[i,:maxlend]
        y = Xtr[i,maxlend:]
        yy = Ytr[i,:]
        yy = np.where(yy)[1]
        prt('L',yy)
        prt('H',y)
        if maxlend:
            prt('D',x)

In [53]:
test_gen(gen(X_train, Y_train, batch_size=batch_size))

L:
Kim
Jong
Un
celebrates
his
birthday
with
a
bang
as
he
seeks
to
cement
rule
~
_
_
_
_
_
_
_
_
_

H:
~
Kim
Jong
Un
celebrates
his
birthday
with
a
bang
as
he
seeks
to
cement
rule
_
_
_
_
_
_
_
_
_

D:
ambitious
blueprint
for
hastening
final
victory
for
our
revolution.
”
Some
analysts
expect
the
regime
to
revise
the
party
charter,
the
organizing
document
of
North

L:
Exclusive:
Lonza
says
in
talks
to
buy
Capsugel;
deal
could
top
$5
billion
-
sources
~
_
_
_
_
_
_
_
_
_

H:
~
Exclusive:
Lonza
says
in
talks
to
buy
Capsugel;
deal
could
top
$5
billion
-
sources
_
_
_
_
_
_
_
_
_

D:
over
100
countries.
Reuters
reported
in
March
that
KKR
was
preparing
to
run
a
sale
process
for
Capsugel
this
year,
and
also
explore
the
possibility

L:
Revisiting
Ferguson:
2
Years
After
The
Shooting
Of
Michael
Brown
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
Revisiting
Ferguson:
2
Years
After
The
Shooting
Of
Michael
Brown
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
of
the
town’s
21,
000
residents,
and
it
often
shifts
to
how
hard


test fliping

In [54]:
test_gen(gen(X_train, Y_train, nflips=6, model=model, debug=False, batch_size=batch_size))

L:
Kim
Jong
Un
celebrates
his
birthday
with
a
bang
as
he
seeks
to
cement
rule
~
_
_
_
_
_
_
_
_
_

H:
~
Kim
Jong
Un
celebrates
his
birthday
with
Usidore.
Usidore.
Usidore.
he
seeks
to
Usidore.
rule
_
_
_
_
_
_
_
_
_

D:
ambitious
blueprint
for
hastening
final
victory
for
our
revolution.
”
Some
analysts
expect
the
regime
to
revise
the
party
charter,
the
organizing
document
of
North

L:
Exclusive:
Lonza
says
in
talks
to
buy
Capsugel;
deal
could
top
$5
billion
-
sources
~
_
_
_
_
_
_
_
_
_

H:
~
Exclusive:
Lonza
says
Jassim
Jassim
to
buy
Capsugel;
deal
Jassim
top
$5
ambivalent.
-
sources
_
_
_
_
_
_
_
_
_

D:
over
100
countries.
Reuters
reported
in
March
that
KKR
was
preparing
to
run
a
sale
process
for
Capsugel
this
year,
and
also
explore
the
possibility

L:
Revisiting
Ferguson:
2
Years
After
The
Shooting
Of
Michael
Brown
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
Revisiting
Ferguson:
2
Jassim
After
The
Shooting
Of
Michael
Brown
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
of
the
town’s
21,
000
residents,
an

In [55]:
valgen = gen(X_test, Y_test,nb_batches=3, batch_size=batch_size)

check that valgen repeats itself after nb_batches

In [56]:
for i in range(4):
    test_gen(valgen, n=1)

L:
Fallen
smartphone
brand
Nokia
challenges
Apple,
Samsung
again
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
Fallen
smartphone
brand
Nokia
challenges
Apple,
Samsung
again
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
long
term,”
Seiche
said,
adding
that
HMD
had
already
set
up
offices
in
40
locations
around
the
world.
NO
FUNDING
FROM
NOKIA
CORP
HMD
is

L:
It’s
time
to
remove
the
‘natural
born
citizen’
requirement
from
the
Constitution
~
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
It’s
time
to
remove
the
‘natural
born
citizen’
requirement
from
the
Constitution
_
_
_
_
_
_
_
_
_
_
_
_

D:
he
was
born
to
a
U.
S.
citizen,
Mr.
Cruz’s
birthplace
does
not
matter
—
and
should
not
matter.
It
also
should
not
matter
for

L:
After
2016
campaign,
more
Americans
consider
Russia
a
threat:
Reuters/Ipsos
poll
~
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
After
2016
campaign,
more
Americans
consider
Russia
a
threat:
Reuters/Ipsos
poll
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
measure
of
accuracy,
of
3
percentage
points
for
the
entire
sample
and
5
percenta

# Train

In [57]:
history = {}

In [58]:
traingen = gen(X_train, Y_train, batch_size=batch_size, nflips=nflips, model=model)
valgen = gen(X_test, Y_test, nb_batches=nb_val_samples//batch_size, batch_size=batch_size)

In [59]:
r = next(traingen)
r[0].shape, r[1].shape, len(r)

((64, 50), (64, 25, 604571), 2)

In [101]:
for iteration in range(500):
    print('Iteration', iteration)
    h = model.fit_generator(traingen, samples_per_epoch=nb_train_samples,
                        nb_epoch=1, validation_data=valgen, nb_val_samples=nb_val_samples
                           )
    for k,v in h.history.iteritems():
        history[k] = history.get(k,[]) + v
    with open('/output/trainhistory.pkl','wb') as fp:
        pickle.dump(history,fp,-1)
    model.save_weights('/output/data.hdf5', overwrite=True)
    gensamples(batch_size=batch_size)

Iteration 0
Epoch 1/1


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., steps_per_epoch=30000, epochs=1, validation_steps=3000)`
  after removing the cwd from sys.path.
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/threadin

StopIteration: 

# Predict

In [102]:
FN = 'predict'

if your GPU is busy you can use CPU for predictions

In [103]:
import os
os.environ['THEANO_FLAGS'] = 'device=cpu,floatX=float32'

In [104]:
import keras
keras.__version__

'2.0.8'

Generate headlines using the "simple" model from http://arxiv.org/pdf/1512.01712v1.pdf

Use indexing of tokens from vocabulary-embedding this does not clip the indexes of the words to vocab_size.

Use the index of outside words to replace them with several oov words (oov , oov0, oov...) that appear in the same description and headline. This will allow headline generator to replace the oov with the same word in the description

In [105]:
FN0 = 'vocabulary-embedding'

we will generate predictions using the model generated in this notebook

In [106]:
FN1 = 'train'

input data (X) is made from maxlend description words followed by eos followed by headline words followed by eos if description is shorter than maxlend it will be left padded with empty if entire data is longer than maxlen it will be clipped and if it is shorter it will be padded.

labels (Y) are the headline words followed by eos and clipped or padded to maxlenh

In other words the input is made from a maxlend half in which the description is padded from the left and a maxlenh half in which eos is followed by a headline followed by another eos if there is enough space.

The labels match only the second half and the first label matches the eos at the start of the second half (following the description in the first half)

the model parameters should be identical with what used in training but notice that maxlend is flexible

In [107]:
maxlend=50 # 0 - if we dont want to use description at all
maxlenh=25
maxlen = maxlend + maxlenh
rnn_size = 512
rnn_layers = 3  # match FN1
batch_norm=False

the out of the first activation_rnn_size nodes from the top layer will be used for activation and the rest will be used to select predicted word

In [108]:
activation_rnn_size = 40 if maxlend else 0

In [109]:
# training parameters
seed=42
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
batch_size=64

In [110]:
nb_train_samples = 30000
nb_val_samples = 3000

# read word embedding

In [111]:
import pickle as pickle

with open('/token/token2.pkl', 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
vocab_size, embedding_size = embedding.shape

In [112]:
nb_unknown_words = 10

In [113]:
print('dimension of embedding space for words',embedding_size)
print('vocabulary size', vocab_size, 'the last {} words can be used as place holders for unknown/oov words'.format(nb_unknown_words))
print('total number of different words',len(idx2word), len(word2idx))
print('number of words outside vocabulary which we can substitue using glove similarity', len(glove_idx2idx))
print('number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(glove_idx2idx))

dimension of embedding space for words 100
vocabulary size 604571 the last 10 words can be used as place holders for unknown/oov words
total number of different words 604573 604573
number of words outside vocabulary which we can substitue using glove similarity 5
number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov) -3


In [114]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

In [115]:
for i in range(vocab_size-nb_unknown_words, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [116]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [117]:
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys

In [118]:
def prt(label, x):
    print(label+':',)
    for w in x:
        print(idx2word[w],)
    print()

# Model

In [119]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.layers.core import Lambda
import keras.backend as K

In [120]:
# seed weight initialization
random.seed(seed)
np.random.seed(seed)

In [121]:
regularizer = l2(weight_decay) if weight_decay else None

# #rnn model

start with a stacked LSTM, which is identical to the bottom of the model used in training

In [122]:
rnn_model = Sequential()
rnn_model.add(Embedding(vocab_size, embedding_size,
                        input_length=maxlen,
                        W_regularizer=regularizer, dropout=p_emb, weights=[embedding], mask_zero=True,
                        name='embedding_1'))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True, # batch_norm=batch_norm,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_%d'%(i+1)
                  )
    rnn_model.add(lstm)
    rnn_model.add(Dropout(p_dense, name='dropout_%d'%(i+1)))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(604571, 100, input_length=75, weights=[array([[-..., mask_zero=True, name="embedding_1", embeddings_regularizer=None)`
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, name="lstm_1", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequenc

# #load

use the bottom weights from the trained model, and save the top weights for later

In [123]:
import h5py
def str_shape(x):
    return 'x'.join(map(str,x.shape))

def inspect_model(model):
    print(model.name)
    for i,l in enumerate(model.layers):
        print(i, 'cls=%s name=%s'%(type(l).__name__, l.name))
        weights = l.get_weights()
        for weight in weights:
            print(str_shape(weight),)
        print()

def load_weights(model, filepath):
    """Modified version of keras load_weights that loads as much as it can
    if there is a mismatch between file and model. It returns the weights
    of the first layer in which the mismatch has happened
    """
    print('Loading', filepath, 'to', model.name)
    flattened_layers = model.layers
    with h5py.File(filepath, mode='r') as f:
        # new file format
        layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]

        # we batch weight value assignments in a single backend call
        # which provides a speedup in TensorFlow.
        weight_value_tuples = []
        for name in layer_names:
            print(name)
            g = f[name]
            weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
            if len(weight_names):
                weight_values = [g[weight_name] for weight_name in weight_names]
                try:
                    layer = model.get_layer(name=name)
                except:
                    layer = None
                if not layer:
                    print('failed to find layer', name, 'in model')
                    print('weights', ' '.join(str_shape(w) for w in weight_values))
                    print('stopping to load all other layers')
                    weight_values = [np.array(w) for w in weight_values]
                    break
                symbolic_weights = layer.trainable_weights + layer.non_trainable_weights
                weight_value_tuples += zip(symbolic_weights, weight_values)
                weight_values = None
        K.batch_set_value(weight_value_tuples)
    return weight_values

In [124]:
weights = load_weights(rnn_model, '/output/data.hdf5')

Loading /output/data.hdf5 to sequential_3


OSError: Unable to open file (unable to open file: name = '/output/data.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
[w.shape for w in weights]

# #headline model

A special layer that reduces the input just to its headline part (second half). For each word in this part it concatenate the output of the previous layer (RNN) with a weighted average of the outputs of the description part. In this only the last rnn_size - activation_rnn_size are used from each output. The first activation_rnn_size output is used to computer the weights for the averaging.

In [ ]:
context_weight = K.variable(1.)
head_weight = K.variable(1.)
cross_weight = K.variable(0.)

def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend], X[:,maxlend:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=([2],[2]))
    # make sure we dont use description words that are masked out
    assert mask.ndim == 2
    activation_energies = K.switch(mask[:, None, :maxlend], activation_energies, -1e20)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=([2],[1]))
    return K.concatenate((context_weight*desc_avg_word, head_weight*head_words))


class SimpleContext(Lambda):
    def __init__(self,**kwargs):
        super(SimpleContext, self).__init__(simple_context,**kwargs)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def get_output_shape_for(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [ ]:
model = Sequential()
model.add(rnn_model)

if activation_rnn_size:
    model.add(SimpleContext(name='simplecontext_1'))

In [ ]:
# we are not going to fit so we dont care about loss and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
n = 2*(rnn_size - activation_rnn_size)
n

perform the top dense of the trained model in numpy so we can play around with exactly how it works

In [ ]:
# out very own softmax
def output2probs(output):
    output = np.dot(output, weights[0]) + weights[1]
    output -= output.max()
    output = np.exp(output)
    output /= output.sum()
    return output

In [ ]:
def output2probs1(output):
    output0 = np.dot(output[:n//2], weights[0][:n//2,:])
    output1 = np.dot(output[n//2:], weights[0][n//2:,:])
    output = output0 + output1 # + output0 * output1
    output += weights[1]
    output -= output.max()
    output = np.exp(output)
    output /= output.sum()
    return output

# Test

In [ ]:
def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

In [ ]:
samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [ ]:
np.all(data[:,maxlend] == eos)

In [ ]:
data.shape,map(len, samples)

In [ ]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

# Sample generation

In [ ]:
# variation to https://github.com/ryankiros/skip-thoughts/blob/master/decoding/search.py
def beamsearch(predict, start=[empty]*maxlend + [eos], avoid=None, avoid_score=1,
               k=1, maxsample=maxlen, use_unk=True, oov=vocab_size-1, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n different elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in xrange(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_samples = []
    dead_scores = []
    live_samples = [list(start)]
    live_scores = [0]

    while live_samples:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)
        assert vocab_size == probs.shape[1]

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk and oov is not None:
            cand_scores[:,oov] = 1e20
        if avoid:
            for a in avoid:
                for i, s in enumerate(live_samples):
                    n = len(s) - len(start)
                    if n < len(a):
                        # at this point live_sample is before the new word,
                        # which should be avoided, is added
                        cand_scores[i,a[n]] += avoid_score
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        dead_scores = [dead_scores[r] for r in ranks if r < n]
        dead_samples = [dead_samples[r] for r in ranks if r < n]
        
        live_scores = [live_scores[r-n] for r in ranks if r >= n]
        live_samples = [live_samples[(r-n)//vocab_size]+[(r-n)%vocab_size] for r in ranks if r >= n]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        def is_zombie(s):
            return s[-1] == eos or len(s) > maxsample
        
        # add zombies to the dead
        dead_scores += [c for s, c in zip(live_samples, live_scores) if is_zombie(s)]
        dead_samples += [s for s in live_samples if is_zombie(s)]
        
        # remove zombies from the living 
        live_scores = [c for s, c in zip(live_samples, live_scores) if not is_zombie(s)]
        live_samples = [s for s in live_samples if not is_zombie(s)]

    return dead_samples, dead_scores

In [ ]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = map(len, samples)
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([output2probs(prob[sample_length-maxlend-1]) for prob, sample_length in zip(probs, sample_lengths)])

In [ ]:
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < vocab_size-nb_unknown_words else glove_idx2idx.get(x,x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= vocab_size-nb_unknown_words])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    return xs

In [ ]:
def vocab_unfold(desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= vocab_size-nb_unknown_words:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]

In [ ]:
import sys
import Levenshtein

def gensamples(X=None, X_test=None, Y_test=None, avoid=None, avoid_score=1, skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    if X is None or isinstance(X,int):
        if X is None:
            i = random.randint(0,len(X_test)-1)
        else:
            i = X
        print('HEAD %d:'%i,' '.join(idx2word[w] for w in Y_test[i]))
        print('DESC:',' '.join(idx2word[w] for w in X_test[i]))
        sys.stdout.flush()
        x = X_test[i]
    else:
        x = [word2idx[w.rstrip('^')] for w in X.split()]
        
    if avoid:
        # avoid is a list of avoids. Each avoid is a string or list of word indeicies
        if isinstance(avoid,str) or isinstance(avoid[0], int):
            avoid = [avoid]
        avoid = [a.split() if isinstance(a,str) else a for a in avoid]
        avoid = [vocab_fold([w if isinstance(w,int) else word2idx[w] for w in a])
                 for a in avoid]

    print('HEADS:')
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, avoid=avoid, avoid_score=avoid_score,
                                   k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in zip(sample,score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code,c) for c in codes])
            if distance > -0.6:
                print(score, ' '.join(words))
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print(score, ' '.join(words))
        codes.append(code)
    return samples

In [ ]:
seed = 8
random.seed(seed)
np.random.seed(seed)

In [ ]:
X = "* Billy Joel is looking for a buyer in Sagaponack^ . Now that he and wife Katie Lee Joel are splitting up , the singer is planning to sell the two oceanfront^ properties he bought for her in 2007 . The four-bedroom mansion ( No . 1 ) and smaller beach bungalow^ ( No . 2 ) will be listed with Corcoran 's Biana^ Stepanian^ for a combined $ 35 million . * Richard Bressler^ , the former CFO of Viacom and now a managing"
Y = "Billy Joel Lists in Sagaponack^"

In [ ]:
samples = gensamples(X=X, skips=2, batch_size=batch_size, k=10, temperature=1.)

In [ ]:
X = "18 Cake GIFs That 'll Make You Moist"
Y = "Is it 350degF^ in here or is it just me ?"

In [ ]:
samples = gensamples(X, skips=2, batch_size=batch_size, k=10, temperature=1.)

In [ ]:
X = "President Barack Obama 's re-election campaign is fundraising off of comments on Obama 's birth certificate by Mitt Romney 's son Matt ."

In [ ]:
gensamples(X, skips=2, batch_size=batch_size, k=10, temperature=1, use_unk=True, short=False);

In [ ]:
X = "What have you been listening to this year ? If you want to find out using cold , hard evidence , then Spotify 's new Year in Music tool will tell you ."
Y = "Spotify Will Make You Smarter for Your App"

In [ ]:
samples = gensamples(X, skips=2, batch_size=batch_size, k=10, temperature=1)

In [ ]:
headline = samples[0][0][len(samples[0][1]):]

In [ ]:
' '.join(idx2word[w] for w in headline)

In [ ]:
avoid = headline

In [ ]:
samples = gensamples(X, avoid=avoid, avoid_score=.1, skips=2, batch_size=batch_size, k=10, temperature=1.)

In [ ]:
avoid = samples[0][0][len(samples[0][1]):]

In [ ]:
samples = gensamples(X, avoid=avoid, avoid_score=.1, skips=2, batch_size=batch_size, k=10, temperature=1.)

In [ ]:
len(samples)

# Weights

In [ ]:
def wsimple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend], X[:,maxlend:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=([2],[2]))
    # make sure we dont use description words that are masked out
    assert mask.ndim == 2
    activation_energies = K.switch(mask[:, None, :maxlend], activation_energies, -1e20)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    return activation_weights


class WSimpleContext(Lambda):
    def __init__(self):
        super(WSimpleContext, self).__init__(wsimple_context)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def get_output_shape_for(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [ ]:
wmodel = Sequential()
wmodel.add(rnn_model)

In [ ]:
wmodel.add(WSimpleContext())

In [ ]:
wmodel.compile(loss='categorical_crossentropy', optimizer=optimizer)

## test

In [ ]:
seed = 8
random.seed(seed)
np.random.seed(seed)

In [ ]:
context_weight.set_value(np.float32(1.))
head_weight.set_value(np.float32(1.))

In [ ]:
X = "Representatives of the groups depicted in The Revenant^ spoke with BuzzFeed News about the actor 's Golden Globes speech calling on listeners to `` protect ... indigenous lands . ''"
Y = "Native American Groups Officially Respond To Leonardo DiCaprio 's Call To Action"

In [ ]:
samples = gensamples(X, skips=2, batch_size=batch_size, k=10, temperature=1.)

In [ ]:
sample = samples[0][0]

In [ ]:
' '.join([idx2word[w] for w in sample])

In [ ]:
data = sequence.pad_sequences([sample], maxlen=maxlen, value=empty, padding='post', truncating='post')
data.shape

In [ ]:
weights = wmodel.predict(data, verbose=0, batch_size=1)
weights.shape

In [ ]:
startd = np.where(data[0,:] != empty)[0][0]
lenh = np.where(data[0,maxlend+1:] == eos)[0][0]
startd, lenh

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(np.array(weights[0,:lenh,startd:].flatten()+1), bins=100);

In [ ]:
import numpy as np
from IPython.core.display import display, HTML

def heat(sample,weights,dark=0.3):
    weights = (weights - weights.min())/(weights.max() - weights.min() + 1e-4)
    html = ''
    fmt = ' <span style="background-color: #{0:x}{0:x}ff">{1}</span>'
    for t,w in zip(sample,weights):
        c = int(256*((1.-dark)*(1.-w)+dark))
        html += fmt.format(c,idx2word[t])
    display(HTML(html))

In [ ]:
heat(sample, weights[0,-1])

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
columns = [idx2word[data[0,i]] for i in range(startd,maxlend)]
rows = [idx2word[data[0,i]] for i in range(maxlend+1,maxlend+lenh+1)]

In [ ]:
df = pd.DataFrame(weights[0,:lenh,startd:],columns=columns,index=rows)

In [ ]:
sns.heatmap(df);